In [1]:
from torch.utils.data import DataLoader
from torch import optim
from torch import nn

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision
import torch
import cv2

import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

BATCH_SIZE = 40
EPOCH = 100
device = torch.device('mps')

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <44B645FB-F027-3EE5-86D7-DBF8E2FC6264> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
transform = transforms.Compose([transforms.Resize(128), transforms.ToTensor()])

trainset = datasets.STL10(root='../data/', split='train', download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = datasets.STL10(root='../data/', split='test', download=True, transform=transform)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)
# model.load_state_dict(
#     torch.load('../models/stl10_resnet18.pth')
# )

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [5]:
for epoch in range(EPOCH + 1):
    running_loss, test_loss = 0.0, 0.0
    for data in tqdm(trainloader, desc=f'{epoch}th Training'):
        image, label = data[0].to(device), data[1].to(device)
        output = model(image)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            for data in testloader:
                test_image, test_label = data[0].to(device), data[1].to(device)
                output = model(test_image)
                loss = criterion(output, test_label)
                test_loss += loss.item()
        model.train()
    
    print(f"Epoch: {epoch}\tLoss : {running_loss / len(trainloader)}\tTest Loss : {test_loss / len(testloader)}")

0th Training: 100%|██████████| 125/125 [41:24<00:00, 19.88s/it]


Epoch: 0	Loss : 1.7297169771194458	Test Loss : 368.7901756989956


1th Training:  71%|███████   | 89/125 [1:45:21<29:31, 49.22s/it]   

In [5]:
def accuracyExprot(dataloader):
    correct, total = 0, 0
    model.eval()
    
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.detach(), 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print("Accuracy: %d%%" % (100 * correct / total))

accuracyExprot(trainloader)
accuracyExprot(testloader)

Accuracy: 9%
Accuracy: 9%


In [6]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [7]:
def cam(dataset, img_sample, img_size):
    tmp = 0
    model.eval()
    with torch.no_grad():
        model.layer4[1].bn2.register_forward_hook(get_activation('final'))
        data, label = dataset[img_sample]
        data.unsqueeze_(0)
        output = model(data.to(device))
        _, prediction = torch.max(output, 1)
        act = activation['final'].squeeze()
        w = model.fc.weight
        for idx in range(act.size(0)):
            if idx == 0:
                tmp = act[idx] * w[prediction.item()][idx]

            else:
                tmp += act[idx] * w[prediction.item()][idx]

            normalization_cam = tmp.cpu().numpy()
            normalization_cam = (normalization_cam - np.min(normalization_cam)) / (np.max(normalization_cam) - np.min(normalization_cam))
            original_img = np.uint8((data[0][0] / 2 + 0.5) * 255)
            cam_img = cv2.resize(np.uint8(normalization_cam * 255), dsize=(img_size, img_size))
    return cam_img, original_img, prediction, label